<a href="https://colab.research.google.com/github/fadilaananta/netflix_recommender/blob/main/Netflix_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import General Libraries and Datasets**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
netflix = pd.read_csv('/content/drive/MyDrive/Dataset/Netflix/netflix_titles.csv')
imdb = pd.read_csv('/content/drive/MyDrive/Dataset/Netflix/MoviesOnStreamingPlatforms_updated.csv')
netflix.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


**Preprocessing the dataset**

In [ ]:
netflix.drop(['show_id', 'listed_in'], axis=1, inplace=True)

In [ ]:
imdb.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
imdb.drop(['Netflix', 'Hulu', 'Prime Video', 'Disney+', 'Type'], axis=1, inplace=True)

In [ ]:
netflix.rename(columns={'title':'Title', 'release_year':'Year', 'director':'Directors', 'country':'Country'}, inplace=True)

In [ ]:
netflix.head()

,type,title,director,cast,country,date_added,release_year,rating,duration,description
0,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,Before planning an awesome wedding for his gra...
1,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Jandino Asporaat riffs on the challenges of ra...
2,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,"With the help of three human allies, the Autob..."
3,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,When a prison ship crash unleashes hundreds of...
4,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,When nerdy high schooler Dani finally attracts...


In [ ]:
netflix.isna().sum()

type               0
title              0
director        1969
cast             570
country          476
date_added        11
release_year       0
rating            10
duration           0
description        0
dtype: int64

In [ ]:
imdb.columns

Index(['Unnamed: 0', 'ID', 'Title', 'Year', 'Age', 'IMDb', 'Rotten Tomatoes',
       'Netflix', 'Hulu', 'Prime Video', 'Disney+', 'Type', 'Directors',
       'Genres', 'Country', 'Language', 'Runtime'],
      dtype='object')

In [ ]:
imdb.isna().sum()

Unnamed: 0             0
ID                     0
Title                  0
Year                   0
Age                 9390
IMDb                 571
Rotten Tomatoes    11586
Netflix                0
Hulu                   0
Prime Video            0
Disney+                0
Type                   0
Directors            726
Genres               275
Country              435
Language             599
Runtime              592
dtype: int64

In [ ]:
imdb.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0


In [ ]:
imdb.set_index('ID', inplace=True)

In [ ]:
print(imdb.shape)
print(netflix.shape)

(16744, 10)
(6234, 10)


**Merge both datasets**

In [ ]:
gabung = pd.merge(netflix.drop_duplicates(), imdb, how='inner', on='Title')
gabung.drop(['Directors_x', 'Year_x', 'Country_y'], axis=1, inplace=True)
gabung.rename(columns={'Directors_y':'Directors', 'Country_x':'Country', 'Year_y':'Year'}, inplace=True)
gabung.head()

,type,Title,cast,Country,date_added,rating,duration,description,Unnamed: 0,ID,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Language,Runtime
0,Movie,Norm of the North: King Sized Adventure,"Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",TV-PG,90 min,Before planning an awesome wedding for his gra...,3074,3075,2019,NaN,3.1,NaN,1,0,0,0,0,"Richard Finn,Tim Maltby","Animation,Adventure,Comedy,Family",English,90.0
1,Movie,#realityhigh,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",TV-14,99 min,When nerdy high schooler Dani finally attracts...,1698,1699,2017,NaN,5.2,NaN,1,0,0,0,0,Fernando Lebrija,"Comedy,Drama,Romance",English,99.0
2,Movie,Automata,"Antonio Banderas, Dylan McDermott, Melanie Gri...","Bulgaria, United States, Spain, Canada","September 8, 2017",R,110 min,"In a dystopian future, an insurance adjuster f...",917,918,2014,18+,6.1,30%,1,0,0,0,0,Gabe Ibáñez,"Action,Drama,Sci-Fi,Thriller",English,109.0
3,Movie,Fabrizio Copano: Solo pienso en mi,Fabrizio Copano,Chile,"September 8, 2017",TV-MA,60 min,Fabrizio Copano takes audience participation t...,3388,3389,2017,NaN,4.9,NaN,1,0,0,0,0,"Francisco Schultz,Rodrigo Toro Solovera",Comedy,Spanish,NaN
4,Movie,Good People,"James Franco, Kate Hudson, Tom Wilkinson, Omar...","United States, United Kingdom, Denmark, Sweden","September 8, 2017",R,90 min,A struggling couple can't believe their luck w...,1851,1852,2014,18+,5.5,9%,1,0,0,0,0,Henrik Ruben Genz,"Action,Crime,Thriller",English,90.0


In [ ]:
gabung.Genres = gabung.Genres.str.replace('-', '')

In [ ]:
gabung.head()

,type,Title,cast,Country,date_added,rating,duration,description,Unnamed: 0,ID,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Language,Runtime
0,Movie,Norm of the North: King Sized Adventure,"Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",TV-PG,90 min,Before planning an awesome wedding for his gra...,3074,3075,2019,NaN,3.1,NaN,1,0,0,0,0,"Richard Finn,Tim Maltby","Animation,Adventure,Comedy,Family",English,90.0
1,Movie,#realityhigh,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",TV-14,99 min,When nerdy high schooler Dani finally attracts...,1698,1699,2017,NaN,5.2,NaN,1,0,0,0,0,Fernando Lebrija,"Comedy,Drama,Romance",English,99.0
2,Movie,Automata,"Antonio Banderas, Dylan McDermott, Melanie Gri...","Bulgaria, United States, Spain, Canada","September 8, 2017",R,110 min,"In a dystopian future, an insurance adjuster f...",917,918,2014,18+,6.1,30%,1,0,0,0,0,Gabe Ibáñez,"Action,Drama,SciFi,Thriller",English,109.0
3,Movie,Fabrizio Copano: Solo pienso en mi,Fabrizio Copano,Chile,"September 8, 2017",TV-MA,60 min,Fabrizio Copano takes audience participation t...,3388,3389,2017,NaN,4.9,NaN,1,0,0,0,0,"Francisco Schultz,Rodrigo Toro Solovera",Comedy,Spanish,NaN
4,Movie,Good People,"James Franco, Kate Hudson, Tom Wilkinson, Omar...","United States, United Kingdom, Denmark, Sweden","September 8, 2017",R,90 min,A struggling couple can't believe their luck w...,1851,1852,2014,18+,5.5,9%,1,0,0,0,0,Henrik Ruben Genz,"Action,Crime,Thriller",English,90.0


In [ ]:
gabung.drop(['cast'], axis=1, inplace=True)

In [ ]:
gabung.dropna(subset=['Genres'], inplace=True)

**Genre-based Recommender**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
genre = cv.fit_transform(gabung['Genres'])

In [ ]:
gabung[gabung.Genres.isna()]

,type,Title,Country,date_added,rating,duration,description,Unnamed: 0,ID,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Language,Runtime


In [ ]:
cv.get_feature_names()

['action',
 'adventure',
 'animation',
 'biography',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'family',
 'fantasy',
 'filmnoir',
 'gameshow',
 'history',
 'horror',
 'music',
 'musical',
 'mystery',
 'news',
 'realitytv',
 'romance',
 'scifi',
 'short',
 'sport',
 'talkshow',
 'thriller',
 'war',
 'western']

In [ ]:
genre.toarray()

array([[0, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
df_genre = pd.DataFrame(genre.toarray(), columns=cv.get_feature_names())
df_genre.index = gabung.Title
df_genre.head()

,action,adventure,animation,biography,comedy,crime,documentary,drama,family,fantasy,filmnoir,gameshow,history,horror,music,musical,mystery,news,realitytv,romance,scifi,short,sport,talkshow,thriller,war,western
Title,,,,,,,,,,,,,,,,,,,,,,,,,,,
Norm of the North: King Sized Adventure,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
#realityhigh,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
Automata,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
Fabrizio Copano: Solo pienso en mi,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Good People,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [ ]:
from scipy.spatial.distance import pdist, squareform
jaccard_distances = pdist(df_genre.values, metric='jaccard')

In [ ]:
distance_df = pd.DataFrame(1 - squareform(jaccard_distances), index=df_genre.index, columns= df_genre.index)

In [ ]:
distance_df.tail(10)

Title,Norm of the North: King Sized Adventure,#realityhigh,Automata,Fabrizio Copano: Solo pienso en mi,Good People,Krish Trish and Baltiboy: Part II,Manhattan Romance,Moonwalkers,Rolling Papers,Stonehearst Asylum,The Runner,6 Years,City of Joy,Laddaland,Next Gen,The Most Assassinated Woman in the World,Article 15,Ee Nagaraniki Emaindi,PK,Hard Tide,One Day,One Day,Phobia 2,Shutter,Carrie Pilby,Marc Maron: Too Real,Saawan,The Heretics,The World We Make,Watchman,Black Panther,Mo Gilligan: Momentum,Big Miracle,Amanda Knox,The Debt Collector,Act of Vengeance,Paradise Lost,Gerald's Game,Long Shot,Our Souls at Night,...,Get Smart,Penelope,Pinky Memsaab,The Bye Bye Man,The Calling,The Legend of Michael Mishra,The Trap,The Trust,War,A Billion Colour Story,A Dangerous Woman,A Sort of Family,Billu,John Mulaney: New in Town,Main Hoon Na,Mandobasar Galpo,Metro,Om Shanti Om,Paheli,Phir Bhi Dil Hai Hindustani,Rukh,Sairat,Secret in Their Eyes,The Aerial,The Duchess,The Search for Life in Space,Una,Beyond All Boundaries,Born to Be Free,Deuces,Elaan,Five Nights in Maine,Life 2.0,Salaakhen,The Tenth Man,Toro,Frank and Cindy,Frank and Cindy,Jeremy Scott: The People's Designer,Friends
Title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Elaan,0.000,0.000000,0.40,0.0,1.00,0.0,0.000000,0.250000,0.000000,0.200000,0.0,0.000000,0.0,0.000000,0.125000,0.166667,0.200000,0.000000,0.000000,0.250000,0.000000,0.000000,0.142857,0.200000,0.000000,0.0,0.250000,0.0,0.0,0.333333,0.200000,0.0,0.000,0.250000,0.250000,0.666667,0.000000,0.200000,0.0,0.000000,...,0.200000,0.000000,0.000000,0.142857,0.250000,0.000000,0.200000,0.400000,0.666667,0.0,0.000000,0.500000,0.000000,0.0,0.285714,0.0,0.333333,0.285714,0.000000,0.000,0.000000,0.000000,0.40,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.500000,1.00,0.0,0.0,0.250000,0.000000,0.600000,0.0,0.0,0.0,0.0
Five Nights in Maine,0.000,0.333333,0.25,0.0,0.00,0.0,0.333333,0.000000,0.000000,0.333333,1.0,0.500000,1.0,0.500000,0.000000,0.250000,0.333333,0.500000,0.333333,0.500000,0.500000,0.500000,0.000000,0.000000,0.500000,0.0,0.500000,0.0,1.0,0.000000,0.000000,0.0,0.250,0.000000,0.000000,0.000000,0.500000,0.333333,0.0,0.500000,...,0.000000,0.000000,0.500000,0.200000,0.500000,0.500000,0.333333,0.000000,0.000000,1.0,0.500000,0.333333,0.500000,0.0,0.166667,1.0,0.000000,0.166667,0.200000,0.250,0.500000,0.500000,0.25,0.333333,0.250,0.000000,0.500000,0.000000,0.0,0.333333,0.00,1.0,0.0,0.500000,0.500000,0.200000,0.0,0.0,0.0,0.0
Life 2.0,0.000,0.000000,0.00,0.0,0.00,0.0,0.000000,0.000000,0.333333,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000,0.500000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000,0.500000,0.000000,0.000000,1.0,0.000000,0.00,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.5,0.0
Salaakhen,0.000,0.250000,0.50,0.0,0.25,0.0,0.250000,0.333333,0.000000,0.250000,0.5,0.333333,0.5,0.333333,0.142857,0.200000,0.250000,0.333333,0.250000,0.333333,0.333333,0.333333,0.000000,0.000000,0.333333,0.0,0.333333,0.0,0.5,0.000000,0.250000,0.0,0.200,0.000000,0.333333,0.333333,0.333333,0.250000,0.0,0.333333,...,0.250000,0.000000,0.333333,0.166667,0.333333,0.333333,0.250000,0.000000,0.333333,0.5,0.333333,0.250000,0.333333,0.0,0.333333,0.5,0.000000,0.333333,0.166667,0.200,0.333333,0.333333,0.20,0.250000,0.200,0.000000,0.333333,0.000000,0.0,0.666667,0.25,0.5,0.0,1.000000,0.333333,0.400000,0.0,0.0,0.0,0.0
The Tenth Man,0.000,0.250000,0.20,0.0,0.00,0.0,0.250000,0.000000,0.000000,0.250000,0.5,0.333333,0.5,0.333333,0.000000,0.200000,0.250000,0.333333,0.250000,0.333333,0.333333,0.333333,0.000000,0.000000,0.333333,0.0,0.333333,0.0,0.5,0.000000,0.000000,0.0,0.200,0.000000,0.000000,0.000

In [ ]:
distance_df = distance_df[~distance_df.index.duplicated()]
test_genre = distance_df[distance_df.index.str.contains('Marvel')].index
genre_df = distance_df.reindex(test_genre)
hasil = pd.DataFrame(genre_df.values.reshape(-1, 1), index=genre_df.columns, columns=['score'])
hasil.sort_values(by='score', ascending=False).head()

,score
Title,
LEGO Marvel Super Heroes: Avengers Reassembled!,1.000000
Stretch Armstrong: The Breakout,1.000000
The Spooky Tale of Captain Underpants Hack-a-ween,0.714286
Kung Fu Panda: Secrets of the Scroll,0.714286
Jimmy Neutron: Boy Genius,0.714286


**Content-based Recommender**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(min_df = 2, max_df = 0.7, stop_words='english')
desc = tf.fit_transform(netflix['description'])

In [ ]:
tf.get_feature_names()

['000',
 '007',
 '10',
 '100',
 '10th',
 '11',
 '12',
 '13',
 '14',
 '14th',
 '15',
 '16',
 '16th',
 '17',
 '17th',
 '18',
 '1888',
 '1899',
 '18th',
 '19',
 '1900s',
 '1920s',
 '1921',
 '1930s',
 '1940s',
 '1945',
 '1950s',
 '1960s',
 '1961',
 '1962',
 '1968',
 '1969',
 '1970',
 '1970s',
 '1971',
 '1975',
 '1976',
 '1979',
 '1980',
 '1980s',
 '1981',
 '1983',
 '1984',
 '1985',
 '1986',
 '1988',
 '1989',
 '1990s',
 '1991',
 '1992',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '19th',
 '20',
 '200',
 '2001',
 '2003',
 '2006',
 '2008',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '20s',
 '20th',
 '21st',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '30',
 '300',
 '30s',
 '32',
 '33',
 '34',
 '35',
 '36th',
 '3d',
 '40',
 '400',
 '40s',
 '40th',
 '43',
 '45',
 '47',
 '4k',
 '50',
 '500',
 '501',
 '50th',
 '51',
 '60',
 '60s',
 '70s',
 '75',
 '80s',
 '90',
 '90s',
 '99',
 'aamir',
 'aaron',
 'abandoned',
 'abandons',
 'abbey',
 '

In [ ]:
desc.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
desc_df = pd.DataFrame(desc.toarray(), columns=tf.get_feature_names(), index=netflix.Title)
desc_df.head()

,000,007,10,100,10th,11,12,13,14,14th,15,16,16th,17,17th,18,1888,1899,18th,19,1900s,1920s,1921,1930s,1940s,1945,1950s,1960s,1961,1962,1968,1969,1970,1970s,1971,1975,1976,1979,1980,1980s,...,year,yearning,yearns,years,yeon,yi,yields,yoga,yoko,yoo,york,yorker,yorkshire,young,younger,youngest,youngsters,youth,youthful,youths,youtube,yu,yugo,yuletide,yumeko,yun,yuppie,zac,zach,zack,zany,zealand,zion,zodiac,zoe,zombie,zombies,zone,zoologist,über
Title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Norm of the North: King Sized Adventure,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Jandino: Whatever it Takes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Transformers Prime,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Transformers: Robots in Disguise,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#realityhigh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
# Find similarity between all items
cosine_similarity_array = cosine_similarity(desc_df)
cosine_similarity(desc_df.loc['Friends'].values.reshape(1, -1), 
                  desc_df.loc['#realityhigh'].values.reshape(1, -1))

array([[0.]])

In [ ]:
desc_df = desc_df[~desc_df.index.duplicated()]
test_data = desc_df[desc_df.index.str.contains('Marvel')].index
test_movie = desc_df.reindex(test_data)
test_movie

,000,007,10,100,10th,11,12,13,14,14th,15,16,16th,17,17th,18,1888,1899,18th,19,1900s,1920s,1921,1930s,1940s,1945,1950s,1960s,1961,1962,1968,1969,1970,1970s,1971,1975,1976,1979,1980,1980s,...,year,yearning,yearns,years,yeon,yi,yields,yoga,yoko,yoo,york,yorker,yorkshire,young,younger,youngest,youngsters,youth,youthful,youths,youtube,yu,yugo,yuletide,yumeko,yun,yuppie,zac,zach,zack,zany,zealand,zion,zodiac,zoe,zombie,zombies,zone,zoologist,über
Title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Marvel & ESPN Films Present: 1 of 1: Genesis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LEGO: Marvel Super Heroes: Maximum Overload,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Marvel's Hulk: Where Monsters Dwell,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.200860,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Marvel Super Hero Adventures: Frost Fight!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Marvel's Iron Man & Hulk: Heroes United,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LEGO Marvel Super Heroes: Black Panther,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LEGO Marvel Super Heroes: Guardians of the Galaxy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LEGO Marvel Super Heroes: Avengers Reassembled!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Marvel's The Defenders,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.198911,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
test_mean = test_movie.mean()
test_mean.sort_values(ascending=False).head()

avengers    0.092042
marvel      0.064163
iron        0.059632
plot        0.050190
thanos      0.049040
dtype: float64

In [ ]:
unwatched = desc_df.drop(test_data, axis=0)
similar = cosine_similarity(test_mean.values.reshape(1, -1), unwatched)
similar_df = pd.DataFrame(similar.T, index=unwatched.index, columns=["score"])
similar_df.sort_values("score", ascending=False).head(10)

,score
Title,
Iron Man & Captain America: Heroes United,0.159316
Hulk Vs.,0.119618
Pitaah,0.116128
Heroes Wanted,0.111158
Power Rangers Samurai,0.110686
Power Rangers Ninja Steel,0.107724
Avengers: Infinity War,0.107062
Power Rangers Lost Galaxy,0.104862
Act of Vengeance,0.100943
